In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = f"https://www.fangraphs.com/players/aaron-judge/15640/stats"
response = requests.get(url)

dom = BeautifulSoup(response.content, 'html.parser')

In [4]:
dom.find_all(attrs={'class':'header_item__6AFbi'})

[<div class="header_item__6AFbi header_item__position__tADVT header_is-mobile__HHswX">OF</div>,
 <div class="header_item__6AFbi">Age: <!-- -->32</div>,
 <div class="header_item__6AFbi">Bats/Throws: <!-- -->R<!-- -->/<!-- -->R</div>,
 <div class="header_item__6AFbi">6'7"<!-- --> / <!-- -->282</div>,
 <div class="header_item__6AFbi header_item__dob-age__eTu_r"><div><span>DOB:</span> <!-- -->4/26/1992</div><div class="header_item__age__gKiWC">(<!-- -->32 years, 1 month, 27 days<!-- -->)</div></div>,
 <div class="header_item__6AFbi header_item__position__tADVT header_is-desktop__PVkUu">OF</div>]

In [7]:
dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text

'Bats/Throws: R/R'

In [8]:
dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-1]

'R'

In [9]:
dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-3]

'R'

In [13]:
import pybaseball
player_data = pybaseball.playerid_reverse_lookup([15640], key_type='fangraphs')
player_data['name_first'][0]

'aaron'

In [16]:
def get_player_split_from_fangraphs(player_id):
    player_data = pybaseball.playerid_reverse_lookup([player_id], key_type='fangraphs')
    first_name = player_data['name_first'][0]
    last_name = player_data['name_last'][0]
    url = f"https://www.fangraphs.com/players/{first_name}-{last_name}/{player_id}/stats"
    response = requests.get(url)
    dom = BeautifulSoup(response.content, 'html.parser')
    player_info = {}
    player_info['IDfg'] = player_id
    player_info['Bats'] = dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-3]
    player_info['Throws'] = dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-1]

    return player_info

get_player_split_from_fangraphs(15640)

{'IDfg': 15640, 'Bats': 'R', 'Throws': 'R'}

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pybaseball

def get_player_split_from_fangraphs(player_id):
    try:
        player_data = pybaseball.playerid_reverse_lookup([player_id], key_type='fangraphs')
        first_name = player_data['name_first'].values[0]
        last_name = player_data['name_last'].values[0]
        url = f"https://www.fangraphs.com/players/{first_name}-{last_name}/{player_id}/stats"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data for player {player_id}")
            return None
        
        print(player_id)
        dom = BeautifulSoup(response.content, 'html.parser')
        player_info = {}
        player_info['IDfg'] = player_id
        player_info['Bats'] = dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-3]
        player_info['Throws'] = dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-1]

        return player_info
    
    except Exception as e:
        print(f"Error fetching splits for player {player_id}: {e}")
        return None

def collect_player_info(player_ids):
    all_info = []

    for player_id in player_ids:
        result = get_player_split_from_fangraphs(player_id)
        if result is not None:
            all_info.append(result)

    return pd.DataFrame(all_info)

# 예제 선수 목록 (FanGraphs IDs)
player_ids = [10155, 5417, 13510]  # 필요한 선수들로 대체

# 선수들의 투타 정보를 수집
player_info_df = collect_player_info(player_ids)
player_info_df

,IDfg,Bats,Throws
0,10155,R,R
1,5417,R,R
2,13510,B,R


In [26]:
import pybaseball
import pandas as pd
from bs4 import BeautifulSoup
import requests
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

def get_player_split_from_fangraphs(player_id):
    try:
        player_data = pybaseball.playerid_reverse_lookup([player_id], key_type='fangraphs')
        first_name = player_data['name_first'].values[0]
        last_name = player_data['name_last'].values[0]
        url = f"https://www.fangraphs.com/players/{first_name}-{last_name}/{player_id}/stats"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data for player {player_id}")
            player_info = {}
            player_info['IDfg'] = player_id
            player_info['Bats'] = 'R'
            player_info['Throws'] = 'R'
            all_info.append(player_info)
            return player_info

        print(player_id)
        dom = BeautifulSoup(response.content, 'html.parser')
        player_info = {}
        player_info['IDfg'] = player_id

        player_info['Bats'] = dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-3]
        player_info['Throws'] = dom.find_all(attrs={'class':'header_item__6AFbi'})[2].text[-1]

        return player_info
    except Exception as e:
        print(f"Error fetching splits for player {player_id}: {e}")
        player_info = {}
        player_info['IDfg'] = player_id
        player_info['Bats'] = 'R'
        player_info['Throws'] = 'R'
        return player_info

def collect_player_split(player_ids):
    all_info = []

    with ThreadPoolExecutor() as thread_executor:
        future_to_player = {thread_executor.submit(get_player_split_from_fangraphs, player_id): player_id for player_id in player_ids}
        
        for future in as_completed(future_to_player):
            player_id = future_to_player[future]
            try:
                result = future.result()
                if result is not None:
                    all_info.append(result)
            except Exception as e:
                print(f"Error processing player {player_id}: {e}")

    return pd.DataFrame(all_info)


def bat_recode(year):
    bat_df = pybaseball.batting_stats(year, qual=30)

    # key_mlbam 및 key_bbref 값을 저장할 리스트 생성
    mlbam_keys = []
    bbref_keys = []

    # 각 선수에 대해 key_mlbam 및 key_bbref 값을 도출하여 리스트에 저장
    for index, row in bat_df.iterrows():
        key_mlbam = get_mlbam_key(row['IDfg'])
        key_bbref = get_bbref_key(row['IDfg'])
        mlbam_keys.append(key_mlbam)
        bbref_keys.append(key_bbref)

    # 결과를 데이터프레임에 추가
    bat_df['key_mlbam'] = mlbam_keys
    bat_df['key_bbref'] = bbref_keys

    # 선수들의 스플릿 데이터를 멀티프로세싱을 통해 수집
    splits_df = collect_player_split(list(bat_df['IDfg']))
    splits_df['Bats'] = splits_df['Bats'].apply(split_to_num)
    splits_df['Throws'] = splits_df['Throws'].apply(split_to_num)
    
    bat_df = pd.merge(bat_df, splits_df[['IDfg', 'Bats']], on='IDfg', how='left')
    bat_df['bat_split'] = bat_df['Bats']

    batting_stats_columns = ['IDfg', 'key_mlbam', 'key_bbref', 'Name', 'Team', 'bat_split', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'Spd', 'BsR', 'wFB/C', 'wSL/C', 'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C', 'O-Swing%', 'Z-Swing%', 'Swing%', 
    'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%', 'wFA/C (sc)', 'wFT/C (sc)', 'wFC/C (sc)', 'wFS/C (sc)', 
    'wFO/C (sc)', 'wSI/C (sc)', 'wSL/C (sc)', 'wCU/C (sc)', 'wKC/C (sc)', 'wEP/C (sc)', 'wCH/C (sc)', 'wSC/C (sc)', 'wKN/C (sc)', 'O-Swing% (sc)', 'Z-Swing% (sc)', 'Swing% (sc)', 'O-Contact% (sc)',
    'Z-Contact% (sc)', 'Contact% (sc)', 'Zone% (sc)', 'LD+%', 'GB%+' ,'FB%+', 'HR/FB%+', 'Pull%+', 'Cent%+', 'Oppo%+', 'Soft%+', 'Med%+', 'Hard%+', 'EV', 'LA', 'Barrel%', 'maxEV', 
    'HardHit%', 'CStr%', 'CSW%', 'wCH/C (pi)', 'wCS/C (pi)', 'wCU/C (pi)', 'wFA/C (pi)', 'wFC/C (pi)', 'wFS/C (pi)', 'wKN/C (pi)', 'wSB/C (pi)', 'wSI/C (pi)', 'wSL/C (pi)', 'O-Swing% (pi)', 
    'Z-Swing% (pi)', 'Swing% (pi)', 'O-Contact% (pi)', 'Z-Contact% (pi)', 'Contact% (pi)', 'Zone% (pi)', 'Pace', 'UBR']

    bat_df = bat_df[batting_stats_columns]

    # 열 이름 변경
    new_column_names = {
        'player_id': 'key_mlbam'
    }

    statcast_batter_exitvelo_barrels_data = pybaseball.statcast_batter_exitvelo_barrels(year, 10)
    statcast_batter_exitvelo_barrels_data_columns = ['player_id', 'anglesweetspotpercent', 'ev50', 'fbld', 'max_distance', 'avg_distance', 'avg_hr_distance', 'ev95percent']
    statcast_batter_exitvelo_barrels_data = statcast_batter_exitvelo_barrels_data[statcast_batter_exitvelo_barrels_data_columns]

    statcast_batter_exitvelo_barrels_data.rename(columns=new_column_names, inplace=True)
    bat_df_merge = pd.merge(bat_df, statcast_batter_exitvelo_barrels_data, on='key_mlbam', how='left')

    return bat_df_merge

def split_to_num(split):
    if split == 'R':
        return 0.0
    elif split == 'L':
        return 1.0
    elif split == 'B':
        return 0.5
    
    
# 각 선수에 대해 key_mlbam 값을 도출하는 함수
def get_mlbam_key(player_id):
    try:
        player_data = pybaseball.playerid_reverse_lookup([player_id], key_type='fangraphs')
        key_mlbam = player_data['key_mlbam'].values[0]
        return key_mlbam
    except Exception as e:
        print(f"Error for player_id {player_id}: {e}")
        return None

def get_bbref_key(player_id):
    try:
        player_data = pybaseball.playerid_reverse_lookup([player_id], key_type='fangraphs')
        key_bbref = player_data['key_bbref'].values[0]
        return key_bbref
    except Exception as e:
        print(f"Error for player_id {player_id}: {e}")
        return None

In [27]:
%%time
bat_df = bat_recode(2023)
bat_df.to_excel('test.xlsx')

25764
20123
20503
5361
13624
12533
18401
18345
13611
23697
19755
25878
19352
26288
20043
15491
26289
12916
16578
23667
14344
13510
18360
18314
15998
20036
15640
27506
19556
16252
17678
5417
26294
19522
13613
19612
19197
13419
12927
12161
25931
16556
16930
21479
16997
21534
20529
18568
11579
16426
24770
19709
11477
17350
19599
19287
9218
20202
15986
29695
11493
19950
16939
12552
30116
13621
26121
19290
19953
19251
16505
17919
24610
12144
27534
18882
18839
11609
10155
29766
11615
13301
27478
26299
17452
21636
11038
24617
19354
19363
19452
9777
18577
5827
15362
23003
21454
19470
24618
19566
22275
1032414551

17338
25976
22664
22799
13185
19627
22244
12037
18900
10264
25768
4940
19326
22715
20437
16219
21865
23312
22263
6184
20454
27647
11739
23565
25961
19238
11489
15161
22581
2967
17929
26323
26668
14162
16535
16530
19956
22514
18126
19918
19858
18373
14854
21853
13066
21558
18819
21840
18564
12856
11442
14221
24679
23401
15112
25807
11445
26466
16572
24336
19913
14813
21523
24064
16442
